<a href="https://colab.research.google.com/github/Bianca-Suriano/Bianca-Suriano-/blob/main/Codigo_para_separar_e_nomear_Informes_de_rendimento_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**RESUMO DO CÓDIGO**

Desenvolvido por: Bianca Suriano

Data de alteração: 04/06/2025

Objetivo Principal:
-------------------
Processar automaticamente arquivos PDF de Informes de Rendimentos (como os emitidos por empresas)
para organizar e extrair documentos individuais de beneficiários, utilizando o CNPJ da empresa
fonte pagadora para estruturar a organização dos arquivos.

Funcionalidades-Chave:
----------------------
1. Identificação da Fonte Pagadora:
   - Extrai o CNPJ da empresa diretamente do PDF
   - Consulta a API da Receita WS para obter o nome oficial da empresa

2. Organização Automática de Arquivos:
   - Cria uma pasta estruturada no formato: [CÓDIGO]_[NOME_EMPRESA]_[CNPJ]
   - Exemplo: "3890_EMPRESA XYZ LTDA_12345678000123"

3. Processamento de Beneficiários:
   - Extrai informações de cada beneficiário (nome e CPF/CNPJ) das páginas do PDF
   - Gera um arquivo PDF individual para cada beneficiário
   - Sanitiza nomes para garantir compatibilidade com sistemas de arquivos

4. Geração de Logs:
   - Cria um arquivo CSV com registro completo de todos os beneficiários processados
   - Inclui página de origem e nome do arquivo gerado

Fluxo de Trabalho:
------------------
1. Identifica o CNPJ da fonte pagadora no documento PDF
2. Consulta o nome da empresa na API oficial usando o CNPJ
3. Cria pasta organizadora com base nas informações obtidas
4. Para cada página do PDF:
   a) Extrai dados do beneficiário
   b) Valida CPF/CNPJ
   c) Gera PDF individual
   d) Armazena na pasta criada
5. Gera relatório final em CSV

Dependências:
-------------
- pypdf: Manipulação de arquivos PDF
- requests: Consultas à API da Receita WS
- Bibliotecas padrão: re, os, shutil, time, unicodedata, csv

Cenários de Uso Típicos:
------------------------
- Departamento financeiro de empresas
- Escritórios de contabilidade
- Setores de recursos humanos
- Processamento em massa de informes de rendimentos
- Organização de documentos fiscais

Resultado Final:
----------------
Uma pasta organizada contendo:
- PDFs individuais para cada beneficiário
- Arquivo CSV com metadados do processamento
- Estrutura de pastas padronizada para fácil identificação
"""


In [1]:
pip install requests

In [2]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 5.8 MB/s eta 0:00:00


processo pasta por pasta


In [3]:
from pypdf import PdfReader, PdfWriter
import re
import os
import shutil
import requests
import time
import unicodedata
import csv

def log_message(message):
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}")

def normalize_text(text):
    if not text:
        return ""
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^A-Z0-9\s\-\.\&]', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\bS\.A\.\b|S\.A\b', 'SA', text, flags=re.IGNORECASE)
    text = re.sub(r'\bLTDA\b', 'LTDA', text, flags=re.IGNORECASE)
    text = re.sub(r'\bME\b', 'ME', text, flags=re.IGNORECASE)
    return text

def remove_name_labels(text):
    if not text:
        return ""
    labels = [
        "Nome Completo", "Nome Empresarial", "Nome",
        "Completo", "Empresarial", "Correlato", "Razao Social",
        "Beneficiaria", "Beneficiario", "Fornecedor", "Fornecedora"
    ]
    for label in labels:
        text = re.sub(rf'\b{label}\s*[:|\-]?\s*', '', text, flags=re.IGNORECASE)
        text = re.sub(rf'\s*{label}\s*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^[\s\|\-:]+', '', text)
    text = re.sub(r'[\s\|\-:]+$', '', text)
    return text.strip()

def sanitize_filename(text):
    text = normalize_text(text)
    text = remove_name_labels(text)
    text = re.sub(r'\s{2,}', ' ', text)
    return text.strip()

def extract_beneficiary_from_item2(page_text):
    bloco_item2 = re.search(r'2\.\s+(PESSOA\s+JUR[IÍ]DICA|PESSOA\s+F[IÍ]SICA)[^\n]*\n([^\n]+)\n([^\n]+)', page_text, re.IGNORECASE)
    if bloco_item2:
        tipo, identificador, nome = bloco_item2.groups()
        identificador = identificador.strip()
        nome = nome.strip()
        nome = remove_name_labels(nome)
        id_type = "CPF" if "." in identificador else "CNPJ"
        return normalize_text(nome), identificador, id_type
    return None, None, None

def extract_paying_source_info(page_text):
    cnpj_pattern = r'\b\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}\b'
    cnpj_match = re.search(cnpj_pattern, page_text)
    if not cnpj_match:
        return None, None
    cnpj = cnpj_match.group(0)
    start = max(0, cnpj_match.start() - 100)
    end = min(len(page_text), cnpj_match.end() + 100)
    context = page_text[start:end]
    patterns = [
        r'FONTE\s+PAGADORA\s*[:|\-]?\s*([A-ZÀ-Ü][A-ZÀ-Ü\s\.]{5,})',
        r'Nome\s+Empresarial\s*[:|\-]?\s*([A-ZÀ-Ü][A-ZÀ-Ü\s\.]{5,})',
        r'([A-ZÀ-Ü][A-ZÀ-Ü\s\.]{5,})\s*' + re.escape(cnpj),
    ]
    for pattern in patterns:
        name_match = re.search(pattern, context, re.IGNORECASE)
        if name_match:
            name = name_match.group(1).strip()
            for label in ["FONTE PAGADORA", "Nome Empresarial", "Nome", "CNPJ"]:
                name = re.sub(rf'{label}\s*[:|\-]?\s*', '', name, flags=re.IGNORECASE)
            name = remove_name_labels(name)
            name = re.sub(r'\s{2,}', ' ', name).strip()
            return name, cnpj
    return None, cnpj

def get_company_name_from_cnpj(cnpj, retries=2, delay=1):
    """Consulta nome da empresa via API com tratamento robusto"""
    cnpj_clean = re.sub(r'\D', '', cnpj)
    if not cnpj_clean.isdigit() or len(cnpj_clean) != 14:
        return None

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json'
    }

    for attempt in range(retries):
        try:
            response = requests.get(
                f"https://www.receitaws.com.br/v1/cnpj/{cnpj_clean}",
                headers=headers,
                timeout=10
            )

            if response.status_code == 200:
                data = response.json()
                if data.get('nome'):
                    return normalize_text(data['nome'])
                elif data.get('message'):
                    log_message(f"API retornou mensagem: {data['message']} para CNPJ {cnpj}")
            elif response.status_code == 429:
                log_message("⚠️ Limite de requisições excedido. Aguardando...")
                time.sleep(5)
                continue
        except requests.exceptions.RequestException as e:
            log_message(f"Erro na requisição: {str(e)}")

        if attempt < retries - 1:
            time.sleep(delay)

    return None

def validate_cnpj(cnpj):
    """Valida o formato do CNPJ"""
    if not re.match(r'\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}', cnpj):
        return False
    return True

def validate_cpf(cpf):
    """Valida o formato do CPF"""
    if not re.match(r'\d{3}\.\d{3}\.\d{3}-\d{2}', cpf):
        return False
    return True

def create_organization_folder(input_file, paying_cnpj, base_output_dir):
    """Cria pasta de organização no diretório especificado"""
    base_name = os.path.basename(input_file)
    code_match = re.search(r'^(\d+)', base_name)
    code = code_match.group(1) if code_match else "COD"

    # Tenta obter nome da empresa via API
    company_name = "FONTE_DESCONHECIDA"
    if paying_cnpj and validate_cnpj(paying_cnpj):
        log_message(f"Consultando API para CNPJ: {paying_cnpj}")
        company_name = get_company_name_from_cnpj(paying_cnpj) or "FONTE_DESCONHECIDA"

    safe_company_name = sanitize_filename(company_name)
    safe_company_name = re.sub(r'[^\w\s\-\.]', '', safe_company_name)
    safe_company_name = re.sub(r'\s+', ' ', safe_company_name).strip()
    safe_company_name = safe_company_name[:50]  # Limita tamanho

    # Formata CNPJ (apenas dígitos)
    if paying_cnpj and validate_cnpj(paying_cnpj):
        safe_cnpj = re.sub(r'\D', '', paying_cnpj)
    else:
        safe_cnpj = "CNPJDESCONHECIDO"

    # Cria nome da pasta no formato especificado
    folder_name = f"{code}_{safe_company_name}_{safe_cnpj}"
    folder_name = re.sub(r'[^\w\s\-\.]', '', folder_name)

    # Caminho completo do diretório de saída
    full_output_path = os.path.join(base_output_dir, folder_name)
    os.makedirs(full_output_path, exist_ok=True)

    return full_output_path
######
######
# --- CONFIGURAÇÕES PRINCIPAIS ---
BASE_OUTPUT_DIR = "/content/drive/MyDrive/0. testes/edp/informe de rendimentos 2025"  # Diretório fixo para saída
INPUT_FILENAME = '/content/drive/MyDrive/0. testes/PASRTE2 -INFORMES'
#####
#####

# --- EXECUÇÃO PRINCIPAL ---
log_message(f"Iniciando processamento do arquivo: {INPUT_FILENAME}")

# Verifica e cria diretório de saída se necessário
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)
log_message(f"Diretório de saída configurado: {BASE_OUTPUT_DIR}")

input_pdf = PdfReader(INPUT_FILENAME)
paying_source_cnpj = None

# Identifica CNPJ da fonte pagadora
for page in input_pdf.pages:
    text = page.extract_text() or ""
    _, cnpj = extract_paying_source_info(text)
    if cnpj and validate_cnpj(cnpj):
        paying_source_cnpj = cnpj
        break

if not paying_source_cnpj:
    paying_source_cnpj = "CNPJ_DESCONHECIDO"
    log_message("⚠️ CNPJ da fonte pagadora não identificado no documento")

# Cria pasta com nome obtido via API no diretório especificado
folder_path = create_organization_folder(INPUT_FILENAME, paying_source_cnpj, BASE_OUTPUT_DIR)
log_message(f"Pasta de destino criada: {folder_path}")

# Processa páginas e gera arquivos
generated_files = []
log_data = []

for i, page in enumerate(input_pdf.pages):
    text = page.extract_text() or ""
    name, identifier, id_type = extract_beneficiary_from_item2(text)

    if name and identifier:
        writer = PdfWriter()
        writer.add_page(page)
        safe_name = sanitize_filename(name)
        filename = f"{safe_name}.pdf"
        filename = re.sub(r'[^\w\-\. ]', '', filename)

        # Evita sobreposição de nomes
        counter = 1
        original_filename = filename
        output_path = os.path.join(folder_path, filename)

        while os.path.exists(output_path):
            filename = f"{os.path.splitext(original_filename)[0]}_{counter}.pdf"
            counter += 1
            output_path = os.path.join(folder_path, filename)

        # Salva na pasta de destino
        with open(output_path, "wb") as f:
            writer.write(f)

        generated_files.append(output_path)
        log_data.append([safe_name, identifier, id_type, i+1, filename])
        log_message(f"Página {i+1}: Gerado arquivo para {safe_name} - {filename}")

# Gera log de processamento
log_path = os.path.join(folder_path, "log_beneficiarios.csv")
with open(log_path, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Nome", "Identificador", "Tipo", "Pagina", "Arquivo"])
    writer.writerows(log_data)

log_message(f"Relatório de processamento salvo em: {log_path}")
log_message(f"✅ Processamento concluído com sucesso!")
log_message(f"✅ Total de {len(generated_files)} arquivos gerados")
log_message(f"✅ Pasta de destino: {folder_path}")

[2025-06-06 12:38:43] Iniciando processamento do arquivo: /content/drive/MyDrive/0. testes/PASRTE2 -INFORMES
[2025-06-06 12:38:43] Diretório de saída configurado: /content/drive/MyDrive/0. testes/edp/informe de rendimentos 2025


IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/0. testes/PASRTE2 -INFORMES'

In [6]:
from pypdf import PdfReader, PdfWriter
import re
import os
import shutil
import requests
import time
import unicodedata
import csv
import glob

def log_message(message):
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}")

def normalize_text(text):
    if not text:
        return ""
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^A-Z0-9\s\-\.\&]', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\bS\.A\.\b|S\.A\b', 'SA', text, flags=re.IGNORECASE)
    text = re.sub(r'\bLTDA\b', 'LTDA', text, flags=re.IGNORECASE)
    text = re.sub(r'\bME\b', 'ME', text, flags=re.IGNORECASE)
    return text

def remove_name_labels(text):
    if not text:
        return ""
    labels = [
        "Nome Completo", "Nome Empresarial", "Nome",
        "Completo", "Empresarial", "Correlato", "Razao Social",
        "Beneficiaria", "Beneficiario", "Fornecedor", "Fornecedora"
    ]
    for label in labels:
        text = re.sub(rf'\b{label}\s*[:|\-]?\s*', '', text, flags=re.IGNORECASE)
        text = re.sub(rf'\s*{label}\s*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^[\s\|\-:]+', '', text)
    text = re.sub(r'[\s\|\-:]+$', '', text)
    return text.strip()

def sanitize_filename(text):
    text = normalize_text(text)
    text = remove_name_labels(text)
    text = re.sub(r'\s{2,}', ' ', text)
    return text.strip()

def extract_beneficiary_from_item2(page_text):
    bloco_item2 = re.search(r'2\.\s+(PESSOA\s+JUR[IÍ]DICA|PESSOA\s+F[IÍ]SICA)[^\n]*\n([^\n]+)\n([^\n]+)', page_text, re.IGNORECASE)
    if bloco_item2:
        tipo, identificador, nome = bloco_item2.groups()
        identificador = identificador.strip()
        nome = nome.strip()
        nome = remove_name_labels(nome)
        id_type = "CPF" if "." in identificador else "CNPJ"
        return normalize_text(nome), identificador, id_type
    return None, None, None

def extract_paying_source_info(page_text):
    cnpj_pattern = r'\b\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}\b'
    cnpj_match = re.search(cnpj_pattern, page_text)
    if not cnpj_match:
        return None, None
    cnpj = cnpj_match.group(0)
    start = max(0, cnpj_match.start() - 100)
    end = min(len(page_text), cnpj_match.end() + 100)
    context = page_text[start:end]
    patterns = [
        r'FONTE\s+PAGADORA\s*[:|\-]?\s*([A-ZÀ-Ü][A-ZÀ-Ü\s\.]{5,})',
        r'Nome\s+Empresarial\s*[:|\-]?\s*([A-ZÀ-Ü][A-ZÀ-Ü\s\.]{5,})',
        r'([A-ZÀ-Ü][A-ZÀ-Ü\s\.]{5,})\s*' + re.escape(cnpj),
    ]
    for pattern in patterns:
        name_match = re.search(pattern, context, re.IGNORECASE)
        if name_match:
            name = name_match.group(1).strip()
            for label in ["FONTE PAGADORA", "Nome Empresarial", "Nome", "CNPJ"]:
                name = re.sub(rf'{label}\s*[:|\-]?\s*', '', name, flags=re.IGNORECASE)
            name = remove_name_labels(name)
            name = re.sub(r'\s{2,}', ' ', name).strip()
            return name, cnpj
    return None, cnpj

def get_company_name_from_cnpj(cnpj, retries=2, delay=1):
    """Consulta nome da empresa via API com tratamento robusto"""
    cnpj_clean = re.sub(r'\D', '', cnpj)
    if not cnpj_clean.isdigit() or len(cnpj_clean) != 14:
        return None

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json'
    }

    for attempt in range(retries):
        try:
            response = requests.get(
                f"https://www.receitaws.com.br/v1/cnpj/{cnpj_clean}",
                headers=headers,
                timeout=10
            )

            if response.status_code == 200:
                data = response.json()
                if data.get('nome'):
                    return normalize_text(data['nome'])
                elif data.get('message'):
                    log_message(f"API retornou mensagem: {data['message']} para CNPJ {cnpj}")
            elif response.status_code == 429:
                log_message("⚠️ Limite de requisições excedido. Aguardando...")
                time.sleep(5)
                continue
        except requests.exceptions.RequestException as e:
            log_message(f"Erro na requisição: {str(e)}")

        if attempt < retries - 1:
            time.sleep(delay)

    return None

def validate_cnpj(cnpj):
    """Valida o formato do CNPJ"""
    if not re.match(r'\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}', cnpj):
        return False
    return True

def validate_cpf(cpf):
    """Valida o formato do CPF"""
    if not re.match(r'\d{3}\.\d{3}\.\d{3}-\d{2}', cpf):
        return False
    return True

def create_organization_folder(input_file, paying_cnpj, base_output_dir):
    """Cria pasta de organização no diretório especificado"""
    base_name = os.path.basename(input_file)
    code_match = re.search(r'^(\d+)', base_name)
    code = code_match.group(1) if code_match else "COD"

    # Tenta obter nome da empresa via API
    company_name = "FONTE_DESCONHECIDA"
    if paying_cnpj and validate_cnpj(paying_cnpj):
        log_message(f"Consultando API para CNPJ: {paying_cnpj}")
        company_name = get_company_name_from_cnpj(paying_cnpj) or "FONTE_DESCONHECIDA"

    safe_company_name = sanitize_filename(company_name)
    safe_company_name = re.sub(r'[^\w\s\-\.]', '', safe_company_name)
    safe_company_name = re.sub(r'\s+', ' ', safe_company_name).strip()
    safe_company_name = safe_company_name[:50]  # Limita tamanho

    # Formata CNPJ (apenas dígitos)
    if paying_cnpj and validate_cnpj(paying_cnpj):
        safe_cnpj = re.sub(r'\D', '', paying_cnpj)
    else:
        safe_cnpj = "CNPJDESCONHECIDO"

    # Cria nome da pasta no formato especificado
    folder_name = f"{code}_{safe_company_name}_{safe_cnpj}"
    folder_name = re.sub(r'[^\w\s\-\.]', '', folder_name)

    # Caminho completo do diretório de saída
    full_output_path = os.path.join(base_output_dir, folder_name)
    os.makedirs(full_output_path, exist_ok=True)

    return full_output_path
######
######
# --- CONFIGURAÇÕES PRINCIPAIS ---
# --- CONFIGURAÇÕES PRINCIPAIS ---
INPUT_DIR = '/content/drive/MyDrive/0. testes/PASRTE2 -INFORMES'
BASE_OUTPUT_DIR = "/content/drive/MyDrive/0. testes/informe de rendimentos 2025"

# --- EXECUÇÃO PRINCIPAL ---
log_message(f"Iniciando processamento de arquivos em: {INPUT_DIR}")
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)
log_message(f"Diretório de saída configurado: {BASE_OUTPUT_DIR}")

# Processa todos os arquivos PDF no diretório de entrada
pdf_files = glob.glob(os.path.join(INPUT_DIR, '*.pdf'))
total_files = len(pdf_files)
log_message(f"Encontrados {total_files} arquivos PDF para processamento")

for file_idx, input_file_path in enumerate(pdf_files, 1):
    log_message(f"Processando arquivo {file_idx}/{total_files}: {os.path.basename(input_file_path)}")

    try:
        input_pdf = PdfReader(input_file_path)
        paying_source_cnpj = None

        # Identifica CNPJ da fonte pagadora
        for page in input_pdf.pages:
            text = page.extract_text() or ""
            _, cnpj = extract_paying_source_info(text)
            if cnpj and validate_cnpj(cnpj):
                paying_source_cnpj = cnpj
                break

        if not paying_source_cnpj:
            paying_source_cnpj = "CNPJ_DESCONHECIDO"
            log_message("⚠️ CNPJ da fonte pagadora não identificado no documento")

        # Cria pasta de destino
        folder_path = create_organization_folder(input_file_path, paying_source_cnpj, BASE_OUTPUT_DIR)
        log_message(f"Pasta de destino criada: {folder_path}")

        # Processa páginas e gera arquivos
        generated_files = []
        log_data = []

        for i, page in enumerate(input_pdf.pages):
            text = page.extract_text() or ""
            name, identifier, id_type = extract_beneficiary_from_item2(text)

            if name and identifier:
                writer = PdfWriter()
                writer.add_page(page)
                safe_name = sanitize_filename(name)
                filename = f"{safe_name}.pdf"
                filename = re.sub(r'[^\w\-\. ]', '', filename)

                # Evita sobreposição de nomes
                counter = 1
                original_filename = filename
                output_path = os.path.join(folder_path, filename)

                while os.path.exists(output_path):
                    filename = f"{os.path.splitext(original_filename)[0]}_{counter}.pdf"
                    counter += 1
                    output_path = os.path.join(folder_path, filename)

                # Salva na pasta de destino
                with open(output_path, "wb") as f:
                    writer.write(f)

                generated_files.append(output_path)
                log_data.append([safe_name, identifier, id_type, i+1, filename])
                log_message(f"  Página {i+1}: Gerado arquivo para {safe_name} - {filename}")

        # Gera log de processamento
        log_path = os.path.join(folder_path, "log_beneficiarios.csv")
        with open(log_path, "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Nome", "Identificador", "Tipo", "Pagina", "Arquivo"])
            writer.writerows(log_data)

        log_message(f"✅ Processamento concluído para {os.path.basename(input_file_path)}")
        log_message(f"  Total de {len(generated_files)} arquivos gerados")
        log_message(f"  Pasta de destino: {folder_path}")

    except Exception as e:
        log_message(f"❌ Erro crítico ao processar {input_file_path}: {str(e)}")
        continue

log_message(f"✅ Processamento finalizado! Total de arquivos processados: {len(pdf_files)}")

[2025-06-06 12:43:29] Iniciando processamento de arquivos em: /content/drive/MyDrive/0. testes/PASRTE2 -INFORMES
[2025-06-06 12:43:29] Diretório de saída configurado: /content/drive/MyDrive/0. testes/edp/informe de rendimentos 2025
[2025-06-06 12:43:29] Encontrados 17 arquivos PDF para processamento
[2025-06-06 12:43:29] Processando arquivo 1/17: 5138 - Informe de Rendimentos 2025.pdf
[2025-06-06 12:43:30] Consultando API para CNPJ: 61.562.112/0001-20
[2025-06-06 12:43:30] Pasta de destino criada: /content/drive/MyDrive/0. testes/edp/informe de rendimentos 2025/5138_PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES LTD_61562112000120
[2025-06-06 12:43:30]   Página 1: Gerado arquivo para 61.562.1120001-20 PRICEWATERHOUSECOOPERS AUDITORES IN - 61.562.1120001-20 PRICEWATERHOUSECOOPERS AUDITORES IN.pdf
[2025-06-06 12:43:30]   Página 2: Gerado arquivo para 61.562.1120001-20PRICEWATERHOUSECOOPERS AUDITORES IN - 61.562.1120001-20PRICEWATERHOUSECOOPERS AUDITORES IN.pdf
[2025-06-06 12:43:30] ✅ Pro

Processo em lote

In [18]:
#processo em lote final


from pypdf import PdfReader, PdfWriter
import re
import os
import requests
import time
import unicodedata
import csv
import glob

def log_message(message):
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}")

def normalize_text(text):
    if not text:
        return ""
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^A-Z0-9\s\-\.\&]', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\bS\.A\.\b|S\.A\b', 'SA', text, flags=re.IGNORECASE)
    text = re.sub(r'\bLTDA\b', 'LTDA', text, flags=re.IGNORECASE)
    text = re.sub(r'\bME\b', 'ME', text, flags=re.IGNORECASE)
    text = re.sub(r'\bIN\b', 'INDEPENDENTE', text, flags=re.IGNORECASE)
    return text

def remove_name_labels(text):
    if not text:
        return ""
    labels = [
        "Nome Completo", "Nome Empresarial", "Nome",
        "Completo", "Empresarial", "Correlato", "Razao Social",
        "Beneficiaria", "Beneficiario", "Fornecedor", "Fornecedora",
        "Fonte Pagadora", "Empresa", "Razão Social", "CNPJ", "Pagadora"
    ]
    for label in labels:
        text = re.sub(rf'\b{label}\s*[:|\-]?\s*', '', text, flags=re.IGNORECASE)
        text = re.sub(rf'\s*{label}\s*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^[\s\|\-:]+', '', text)
    text = re.sub(r'[\s\|\-:]+$', '', text)
    return text.strip()

def sanitize_filename(text):
    text = normalize_text(text)
    text = remove_name_labels(text)
    text = re.sub(r'\s{2,}', ' ', text)
    return text.strip()

def extract_beneficiary_info(page_text):
    patterns = [
        r'2\.\s*PESSOA\s+(?:JUR[IÍ]DICA|F[IÍ]SICA).*?FORNECEDORA\s+DO\s+SERVI[CÇ]O\s*.*?CNPJ\s*[\|:]?\s*(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})\s*[\s\S]*?Nome\s+completo\s*[\|:]?\s*([^\n]+)',
        r'2\.\s*PESSOA\s+(?:JUR[IÍ]DICA|F[IÍ]SICA).*?BENEFICIÁRIA\s+[^\n]+\s*[\|:]?\s*Nome\s+Empresarial\s*([^\n]+?)\s+CNPJ\s*(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})',
        r'2\.\s*PESSOA\s+(?:JUR[IÍ]DICA|F[IÍ]SICA)[\s\S]*?(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})[\s\S]*?([A-Z][A-Z\s\.]{10,})'
    ]
    for pattern in patterns:
        match = re.search(pattern, page_text, re.IGNORECASE | re.DOTALL)
        if match:
            if match.lastindex == 2:
                cnpj = match.group(1).strip()
                name = match.group(2).strip()
            else:
                cnpj = match.group(2).strip()
                name = match.group(1).strip()
            name = re.sub(r'[\|\-]', ' ', name)
            name = remove_name_labels(name)
            name = re.sub(r'\s{2,}', ' ', name).strip()
            return normalize_text(name), cnpj, "CNPJ"
    return None, None, None

def extract_paying_source_info(page_text):
    patterns = [
        r'1\.\s*FONTE\s+PAGADORA\s*.*?Nome\s*[\|:]?\s*([^\n]+?)\s*[\s\S]*?CNPJ\s*[\|:]?\s*(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})',
        r'1\.\s*FONTE\s+PAGADORA\s*[\|:]?\s*Nome\s+Empresarial\s*([^\n]+?)\s+CNPJ\s*(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})',
        r'1\.\s*FONTE\s+PAGADORA[\s\S]*?([A-Z][A-Z\s\.]{10,})\s*[\s\S]*?(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})'
    ]
    for pattern in patterns:
        match = re.search(pattern, page_text, re.IGNORECASE | re.DOTALL)
        if match:
            if 'CNPJ' in match.group(0):
                name = match.group(1).strip()
                cnpj = match.group(2).strip()
            else:
                name = match.group(2).strip()
                cnpj = match.group(1).strip()
            name = re.sub(r'[\|\-]', ' ', name)
            name = remove_name_labels(name)
            name = re.sub(r'\s{2,}', ' ', name).strip()
            return name, cnpj
    return None, None

def get_company_name_from_cnpj(cnpj, retries=1, delay=1):
    cnpj_clean = re.sub(r'\D', '', cnpj)
    if not cnpj_clean.isdigit() or len(cnpj_clean) != 14:
        return None
    headers = {'User-Agent': 'Mozilla/5.0'}
    for attempt in range(retries):
        try:
            response = requests.get(f"https://www.receitaws.com.br/v1/cnpj/{cnpj_clean}", headers=headers, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if data.get('nome'):
                    return normalize_text(data['nome'])
            elif response.status_code == 429:
                time.sleep(2)
        except Exception:
            pass
        if attempt < retries - 1:
            time.sleep(delay)
    return None

def validate_cnpj(cnpj):
    return bool(re.match(r'\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}', cnpj))

def create_organization_folder(input_file, paying_source_name, paying_cnpj, base_output_dir):
    base_name = os.path.basename(input_file)
    code_match = re.search(r'^(\d+)', base_name)
    code = code_match.group(1) if code_match else "COD"
    if paying_source_name:
        company_name = paying_source_name
    elif paying_cnpj:
        company_name = get_company_name_from_cnpj(paying_cnpj) or "FONTE_DESCONHECIDA"
    else:
        company_name = "FONTE_DESCONHECIDA"
    safe_company_name = sanitize_filename(company_name)
    safe_company_name = re.sub(r'[^\w\s\-\.]', '', safe_company_name)
    safe_company_name = re.sub(r'\s+', ' ', safe_company_name).strip()
    if len(safe_company_name) > 50:
        safe_company_name = safe_company_name[:45] + "..."
    clean_cnpj = re.sub(r'\D', '', paying_cnpj) if paying_cnpj and validate_cnpj(paying_cnpj) else "CNPJ_NAO_IDENTIFICADO"
    folder_name = f"{code}_{safe_company_name}_{clean_cnpj}"
    folder_name = re.sub(r'[^\w\s\-\.]', '', folder_name)
    folder_name = re.sub(r'\s+', ' ', folder_name).strip()
    full_output_path = os.path.join(base_output_dir, folder_name)
    os.makedirs(full_output_path, exist_ok=True)
    return full_output_path, company_name, clean_cnpj

# --- CONFIGURAÇÕES ---
INPUT_DIR = '/content/drive/MyDrive/0. testes/0./t2'
BASE_OUTPUT_DIR = "/content/drive/MyDrive/0. testes/0./t2"

# --- EXECUÇÃO PRINCIPAL ---
log_message(f"Iniciando processamento de arquivos em: {INPUT_DIR}")
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)
pdf_files = glob.glob(os.path.join(INPUT_DIR, '*.pdf'))
total_files = len(pdf_files)
log_message(f"Encontrados {total_files} arquivos PDF para processamento")

for file_idx, input_file_path in enumerate(pdf_files, 1):
    filename = os.path.basename(input_file_path)
    log_message(f"\n{'='*50}")
    log_message(f"Processando [{file_idx}/{total_files}]: {filename}")
    log_message(f"{'='*50}")
    try:
        pdf = PdfReader(input_file_path)
        paying_source_name = None
        paying_source_cnpj = None
        for i, page in enumerate(pdf.pages):
            text = page.extract_text() or ""
            name, cnpj = extract_paying_source_info(text)
            if cnpj:
                paying_source_cnpj = cnpj
                paying_source_name = name
                log_message(f"  Fonte Pagadora encontrada na página {i+1}")
                break
        if not paying_source_cnpj:
            log_message("⚠️ ATENÇÃO: CNPJ da fonte pagadora não identificado")
            paying_source_cnpj = "CNPJ_NAO_IDENTIFICADO"
        folder_path, company_name, clean_cnpj = create_organization_folder(
            input_file_path,
            paying_source_name,
            paying_source_cnpj,
            BASE_OUTPUT_DIR
        )
        log_message(f"  Nome Fonte: {company_name}")
        log_message(f"  CNPJ Fonte: {clean_cnpj}")
        log_message(f"  Pasta Destino: {os.path.basename(folder_path)}")

        generated_files = []
        log_data = []
        beneficiaries_found = 0

        for i, page in enumerate(pdf.pages):
            text = page.extract_text() or ""
            name, identifier, id_type = extract_beneficiary_info(text)
            if name and identifier:
                beneficiaries_found += 1
                writer = PdfWriter()
                writer.add_page(page)
                safe_name = sanitize_filename(name)
                id_clean = re.sub(r'\D', '', identifier)
                base_filename = f"{safe_name}_{id_clean}.pdf"
                base_filename = re.sub(r'[^\w\-\. ]', '', base_filename)
                output_path = os.path.join(folder_path, base_filename)
                counter = 1
                while os.path.exists(output_path):
                    new_filename = f"{safe_name}_{id_clean}_{counter}.pdf"
                    output_path = os.path.join(folder_path, new_filename)
                    counter += 1
                with open(output_path, "wb") as f:
                    writer.write(f)
                generated_files.append(output_path)
                log_data.append([
                    safe_name,
                    identifier,
                    id_type,
                    i+1,
                    os.path.basename(output_path)
                ])
                log_message(f"  Página {i+1}: Beneficiário '{safe_name}' - CNPJ: {identifier}")

        if log_data:
            log_path = os.path.join(folder_path, "log_beneficiarios.csv")
            with open(log_path, "w", newline="", encoding="utf-8") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(["Nome", "CNPJ", "Tipo", "Pagina", "Arquivo"])
                writer.writerows(log_data)

        log_message(f"  ✅ {beneficiaries_found} beneficiários processados")
        log_message(f"  📂 Pasta: {os.path.basename(folder_path)}")

    except Exception as e:
        log_message(f"  ❌ ERRO CRÍTICO: {str(e)}")
        continue

log_message(f"\n{'='*50}")
log_message(f"✅ PROCESSAMENTO FINALIZADO!")
log_message(f"📊 Total de arquivos processados: {len(pdf_files)}")
log_message(f"{'='*50}")


[2025-06-06 14:18:03] Iniciando processamento de arquivos em: /content/drive/MyDrive/0. testes/0./t2
[2025-06-06 14:18:03] Encontrados 4 arquivos PDF para processamento
[2025-06-06 14:18:03] 
[2025-06-06 14:18:03] Processando [1/4]: 5138 - Informe de Rendimentos 2025.pdf
[2025-06-06 14:18:03] ==================================================
[2025-06-06 14:18:03]   Fonte Pagadora encontrada na página 1
[2025-06-06 14:18:03]   Nome Fonte: CENTRAL GERADORA FOTOVOLTAICA MONTE VERDE SOLAR I S.A.
[2025-06-06 14:18:03]   CNPJ Fonte: 61562112000120
[2025-06-06 14:18:03]   Pasta Destino: 5138_CENTRAL GERADORA FOTOVOLTAICA MONTE VERDE SOL..._61562112000120
[2025-06-06 14:18:03]   Página 1: Beneficiário 'PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTE' - CNPJ: 61.562.112/0001-20
[2025-06-06 14:18:03]   Página 2: Beneficiário 'PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTE' - CNPJ: 61.562.112/0001-20
[2025-06-06 14:18:03]   ✅ 2 beneficiários processados
[2025-06-06 14:18:03]   📂 Pasta: 5138_CENTRAL